- consider only xy in validation

In [1]:
# credit: https://www.kaggle.com/kokitanisaka/lstm-by-keras-with-unified-wi-fi-feats
!pip install tensorflow-determinism

  Created wheel for tensorflow-determinism: filename=tensorflow_determinism-0.1.0-py3-none-any.whl size=5255 sha256=0cf3c6db8208655c2ce1aad4d1ea3b07b57da03295b7880cdc122812cdf3c2b4
  Stored in directory: /root/.cache/pip/wheels/ce/06/d4/df757adc4c81f705a74a0317c27cf19919ccd25ae2a6ecd2c5
Successfully built tensorflow-determinism


In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob
import pickle

import random
import os

from tqdm.notebook import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow_addons.layers import WeightNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *

import warnings
warnings.filterwarnings("ignore")

In [3]:
# options
N_SPLITS = 5
SEED = 2021
NUM_FEATS = 25 # number of features that we use. there are 100 feats but we don't need to use all of them
base_path = '/kaggle'

In [4]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    np.random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

In [5]:
feature_dir = f"{base_path}/input/indooruniteddataset"
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))
subm = pd.read_csv(f'{base_path}/input/indoor-location-navigation/sample_submission.csv', index_col=0)

In [6]:
# training target features
BSSID_FEATS = [f'bssid_{i}' for i in range(NUM_FEATS)]
RSSI_FEATS  = [f'rssi_{i}' for i in range(NUM_FEATS)]

In [7]:
data = []
for i in tqdm(train_files):
    tmp = pd.read_csv(i)
    tmp["site_id"] = i.split("/")[4].split("_")[0]
    data.append(tmp)
data = pd.concat(data)

test_data = []
for i in tqdm(test_files):
    tmp = pd.read_csv(i)
    tmp["site_id"] = i.split("/")[4].split("_")[0]
    test_data.append(tmp)
test_data = pd.concat(test_data)

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [8]:
wifi_feat_nums = 60
wifi_bssids = []
for i in range(wifi_feat_nums):
    wifi_bssids.extend(data.iloc[:,i].values.tolist())
wifi_bssids = list(set(wifi_bssids))

wifi_bssids_size = len(wifi_bssids)
print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids_test = []
for i in range(wifi_feat_nums):
    wifi_bssids_test.extend(test_data.iloc[:,i].values.tolist())
wifi_bssids_test = list(set(wifi_bssids_test))

wifi_bssids_size = len(wifi_bssids_test)
print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids.extend(wifi_bssids_test)
wifi_bssids_size = len(wifi_bssids)

BSSID TYPES: 54067
BSSID TYPES: 25654


In [9]:
# preprocess

le = LabelEncoder()
le.fit(wifi_bssids)
le_site = LabelEncoder()
le_site.fit(data['site_id'])

ss = StandardScaler()
ss.fit(data.loc[:,RSSI_FEATS])

StandardScaler()

In [10]:
data.loc[:,RSSI_FEATS] = ss.transform(data.loc[:,RSSI_FEATS])
for i in BSSID_FEATS:
    data.loc[:,i] = le.transform(data.loc[:,i])
    data.loc[:,i] = data.loc[:,i] + 1
    
data.loc[:, 'site_id'] = le_site.transform(data.loc[:, 'site_id'])

data.loc[:,RSSI_FEATS] = ss.transform(data.loc[:,RSSI_FEATS])

In [11]:
test_data.loc[:,RSSI_FEATS] = ss.transform(test_data.loc[:,RSSI_FEATS])
for i in BSSID_FEATS:
    test_data.loc[:,i] = le.transform(test_data.loc[:,i])
    test_data.loc[:,i] = test_data.loc[:,i] + 1
    
test_data.loc[:, 'site_id'] = le_site.transform(test_data.loc[:, 'site_id'])

test_data.loc[:,RSSI_FEATS] = ss.transform(test_data.loc[:,RSSI_FEATS])

In [12]:
site_count = len(data['site_id'].unique())
data.reset_index(drop=True, inplace=True)

In [13]:
set_seed(SEED)

# model

In [14]:
num_heads = 8
bssid_embed_dim = 64
site_embed_dim = 2
ff_dim = 128 
DROPOUT = 0.2
maxlen = NUM_FEATS

In [15]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=DROPOUT):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [16]:
def create_model(input_data):

    # bssid feats
    input_dim = input_data[0].shape[1]

    input_embd_layer = L.Input(shape=(input_dim,))
    x1 = L.Embedding(wifi_bssids_size, bssid_embed_dim)(input_embd_layer)
    #x1 = TokenAndPositionEmbedding(maxlen, wifi_bssids_size, bssid_embed_dim)(input_embd_layer)
    #x1 = TransformerBlock(bssid_embed_dim, num_heads, ff_dim)(x1)
    #x1 = LSTM(128)(x1)
    x1 = L.Flatten()(x1)

    # rssi feats
    input_dim = input_data[1].shape[1]

    input_layer = L.Input(input_dim, )
    x2 = L.BatchNormalization()(input_layer)
    x2 = L.Dense(NUM_FEATS * 64, activation='relu')(x2)

    # site
    input_site_layer = L.Input(shape=(1,))
    x3 = L.Embedding(site_count, 2)(input_site_layer)
    #x3 = TokenAndPositionEmbedding(1, site_count, bssid_embed_dim)(input_site_layer)
    #x3 = TransformerBlock(embed_dim, num_heads, ff_dim)(x3)
    #x3 = TransformerBlock(embed_dim, num_heads, ff_dim)(x3)
    #x3 = TransformerBlock(embed_dim, num_heads, ff_dim)(x3)
    #x3 = LSTM(1)(x3)
    x3 = L.Flatten()(x3)
    
    # main stream
    x = L.Concatenate(axis=1)([x1, x3, x2])

    x = L.BatchNormalization()(x)
    x = L.Dropout(0.3)(x)
    x = L.Dense(256, activation='relu')(x)

    x = L.Reshape((1, -1))(x)
    x = L.BatchNormalization()(x)
    x = L.LSTM(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=True, activation='relu')(x)
    x = L.LSTM(16, dropout=0.1, return_sequences=False, activation='relu')(x)
    
    output_layer_1 = L.Dense(2, name='xy')(x)
    output_layer_2 = L.Dense(1, activation='softmax', name='floor')(x)

    model = M.Model([input_embd_layer, input_layer, input_site_layer], 
                    [output_layer_1, output_layer_2])

    model.compile(optimizer=tf.optimizers.Adam(lr=0.001),
                  loss='mse', metrics=['mse'])

    return model

In [17]:
batch_size=256
epochs=30

score_df = pd.DataFrame()
oof = list()
predictions = list()

oof_x, oof_y, oof_f = np.zeros(data.shape[0]), np.zeros(data.shape[0]), np.zeros(data.shape[0])
preds_x, preds_y = 0, 0
preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))

for fold, (trn_idx, val_idx) in enumerate(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED).split(data.loc[:, 'path'], data.loc[:, 'path'])):
    X_train = data.loc[trn_idx, BSSID_FEATS + RSSI_FEATS + ['site_id']]
    y_trainx = data.loc[trn_idx, 'x']
    y_trainy = data.loc[trn_idx, 'y']
    y_trainf = data.loc[trn_idx, 'floor']

    tmp = pd.concat([y_trainx, y_trainy], axis=1)
    y_train = [tmp, y_trainf]

    X_valid = data.loc[val_idx, BSSID_FEATS + RSSI_FEATS + ['site_id']]
    y_validx = data.loc[val_idx, 'x']
    y_validy = data.loc[val_idx, 'y']
    y_validf = data.loc[val_idx, 'floor']

    tmp = pd.concat([y_validx, y_validy], axis=1)
    y_valid = [tmp, y_validf]

    model = create_model([X_train.loc[:,BSSID_FEATS], X_train.loc[:,RSSI_FEATS], X_train.loc[:,'site_id']])
    model.fit([X_train.loc[:,BSSID_FEATS], X_train.loc[:,RSSI_FEATS], X_train.loc[:,'site_id']], y_train, 
                validation_data=([X_valid.loc[:,BSSID_FEATS], X_valid.loc[:,RSSI_FEATS], X_valid.loc[:,'site_id']], y_valid), 
                batch_size=batch_size, epochs=epochs,
                callbacks=[
                ReduceLROnPlateau(monitor='val_xy_loss', factor=0.1, patience=3, verbose=1, min_delta=1e-4, mode='min')
                , ModelCheckpoint(f'{base_path}/RNN_{SEED}_{fold}.hdf5', monitor='val_xy_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')
                , EarlyStopping(monitor='val_xy_loss', min_delta=1e-4, patience=5, mode='min', baseline=None, restore_best_weights=True)
            ])

    model.load_weights(f'{base_path}/RNN_{SEED}_{fold}.hdf5')
    val_pred = model.predict([X_valid.loc[:,BSSID_FEATS], X_valid.loc[:,RSSI_FEATS], X_valid.loc[:,'site_id']])

    oof_x[val_idx] = val_pred[0][:,0]
    oof_y[val_idx] = val_pred[0][:,1]
    oof_f[val_idx] = val_pred[1][:,0].astype(int)
    
    pred = model.predict([test_data.loc[:,BSSID_FEATS], test_data.loc[:,RSSI_FEATS], test_data.loc[:,'site_id']])
    preds_x += pred[0][:,0]
    preds_y += pred[0][:,1]
    preds_f_arr[:, fold] = pred[1][:,0].astype(int)

    score = comp_metric(oof_x[val_idx], oof_y[val_idx], oof_f[val_idx],
                        y_validx.to_numpy(), y_validy.to_numpy(), y_validf.to_numpy())
    print(f"fold {fold}: mean position error {score}")

    K.clear_session()

preds_x /= (fold + 1)
preds_y /= (fold + 1)
    
print("*+"*40)
# as it breaks in the middle of cross-validation, the score is not accurate at all.
score = comp_metric(oof_x, oof_y, oof_f, data.iloc[:, -5].to_numpy(), data.iloc[:, -4].to_numpy(), data.iloc[:, -3].to_numpy())
oof.append(score)
print(f"mean position error {score}")
print("*+"*40)

preds_f_mode = stats.mode(preds_f_arr, axis=1)
preds_f = preds_f_mode[0].astype(int).reshape(-1)
test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
test_preds.columns = subm.columns
test_preds.index = test_data["site_path_timestamp"]
test_preds["floor"] = test_preds["floor"].astype(int)
predictions.append(test_preds)

Epoch 1/30
1115/1115 [==============================] - 82s 69ms/step - loss: 2993.2188 - xy_loss: 2989.8647 - floor_loss: 3.3529 - xy_mse: 2989.8647 - floor_mse: 3.3529 - val_loss: 85.9396 - val_xy_loss: 82.5918 - val_floor_loss: 3.3479 - val_xy_mse: 82.5918 - val_floor_mse: 3.3479

Epoch 00001: val_xy_loss improved from inf to 82.59177, saving model to /kaggle/RNN_2021_0.hdf5
Epoch 2/30
1115/1115 [==============================] - 79s 71ms/step - loss: 190.6909 - xy_loss: 187.3562 - floor_loss: 3.3346 - xy_mse: 187.3562 - floor_mse: 3.3346 - val_loss: 65.8066 - val_xy_loss: 62.4587 - val_floor_loss: 3.3479 - val_xy_mse: 62.4587 - val_floor_mse: 3.3479

Epoch 00002: val_xy_loss improved from 82.59177 to 62.45872, saving model to /kaggle/RNN_2021_0.hdf5
Epoch 3/30
1115/1115 [==============================] - 78s 70ms/step - loss: 161.3680 - xy_loss: 158.0207 - floor_loss: 3.3473 - xy_mse: 158.0207 - floor_mse: 3.3473 - val_loss: 56.4388 - val_xy_loss: 53.0910 - val_floor_loss: 3.3479 -

# submission

In [18]:
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)

lgb_pred = pd.read_csv('../input/indoorlgbfloor/submission.csv')
all_preds['floor'] = lgb_pred['floor'].values

In [19]:
all_preds.to_csv('submission.csv')